# How controls will work

- engine begins with initial fuel setting and resistor level

### Control States

- lightoff sequencing
  - normal process with initial fuel setting

- manaul mode
  - same as lightoff
  - manual fues settings adjusted

- PID
  - initialize PID with last fuel setting as initial
  - implement ramp to avoid large changes
  - adjust PID from parameter page
  - adjust n2 setpoint from control page


In [ ]:
# listening for n2 setpoint changes

In [1]:
# library imports
from simple_pid import PID  # pid loop for n2


In [8]:
# PID control
# get from redis...
p_p, p_i, p_d = 5, .1, 0
init_ao = 55

# initialize at start of program
n2_pid = PID(p_p, p_i, p_d, setpoint=20)
n2_pid.sample_time = 0.5
n2_pid.output_limits = (0, 100)

# ending or initializing always do!
n2_pid.set_auto_mode(False)

# PID begins
n2_pid.set_auto_mode(True, last_output=init_ao)

# calling pid loop with max change
def call_pid(val, prev_ao):
    prev_o = n2_pid
    new_ao = n2_pid(val)
    delta = new_ao-prev_ao

    # prevent new value from being more than 3
    if new_ao > prev_ao:
        new_ao = min(prev_ao+3, new_ao)
    else:
        new_ao = max(prev_ao-3, new_ao)

    return new_ao

n2_val = 25
fuel_ao = call_pid(n2_val, init_ao)
print(fuel_ao)


52


In [ ]:
# listening for changes to parameters. Put in state machine logic
pub_msg = 'msg'
if 'kp' in pub_msg:
    # set pid values
    kp, ki, kd = pub_msg['kp'], pub_msg['ki'], pub_msg['kd']
    n2_pid.tunings = (kp, ki, kd)
    # set redis values in hash map
    pass

elif 'n2_sp' in pub_msg:
    n2_sp = pub_msg['n2_sp']
    n2_pid.setpoint = n2_sp
    # set on redis

elif 'init_ao' in pub_msg:
    # set on redis
    pass


In [22]:
# n2 simulator speed
def speed_response(fuel_percent, load_level):
    n2_max_sim = 60
    # equation came up with for simulating... not representative
    n2_speed = n2_max_sim * (fuel_percent/100) * ((10/load_level)/2)
    n1_speed = n2_speed * 3
    return round(n1_speed, 1), round(n2_speed, 1)

speed_response(100, 6)

(150.0, 50.0)